In [2]:
from src.misc import gg_authentication, read_ggs, format_sql
import altair as alt
from pyprojroot import here
import duckdb

In [3]:
gg_auth = gg_authentication("./keys/secret_key.json")

In [4]:
data_extraction = read_ggs(
    gg_auth["credentials"],
    "1auVrm09jvShYQxfS3cHV-4SF29ZScqQ1Pc8T4Y4lOe8",
    "data",
)

In [11]:
us_house_price = duckdb.query(
    format_sql(here("./sql/format_data_type.sql"))
).to_df()

In [12]:
us_house_price.describe()

,time,price,homes_sold,new_listings,inventory,days_on_market,avg_sale_to_list
count,1036,1036.000000,1036.000000,1036.000000,1.036000e+03,1036.00000,1036.000000
mean,2018-02-14 20:06:29.189189,396.888996,74148.686293,89414.602317,2.239134e+05,38.80695,0.990241
min,2012-01-01 00:00:00,134.000000,972.000000,1009.000000,1.053000e+03,5.00000,0.921000
25%,2015-01-24 06:00:00,252.750000,3320.000000,3653.750000,7.314500e+03,23.00000,0.977000
50%,2018-02-15 00:00:00,367.000000,5153.500000,6228.500000,1.255500e+04,36.00000,0.990000
75%,2021-03-08 18:00:00,510.000000,7385.750000,9223.500000,2.252650e+04,50.00000,1.002000
max,2024-04-01 00:00:00,900.000000,726239.000000,839201.000000,2.159584e+06,145.00000,1.107000
std,NaN,181.516995,174426.699646,211100.335460,5.406954e+05,21.49035,0.022459


In [14]:
alt.Chart(us_house_price).mark_line().encode(
    x=alt.X("time"),
    y=alt.Y("price"),
    color=alt.Color(
        "region",
        sort=alt.EncodingSortField("price", op="mean", order="descending"),
    ),
)

alt.Chart(...)

In [42]:
price_growth = duckdb.query(
    format_sql(here("./sql/price_growth_yoy.sql"))
).to_df()
price_growth[price_growth["region"] == "Chicago, IL metro area"]

,year,region,price_growth
0,2022-01-01,"Chicago, IL metro area",0.036323
1,2023-01-01,"Chicago, IL metro area",0.027501
2,2024-01-01,"Chicago, IL metro area",0.037523
36,2017-01-01,"Chicago, IL metro area",0.058890
37,2018-01-01,"Chicago, IL metro area",0.037077
38,2019-01-01,"Chicago, IL metro area",0.018563
39,2020-01-01,"Chicago, IL metro area",0.082686
40,2021-01-01,"Chicago, IL metro area",0.115648
80,2013-01-01,"Chicago, IL metro area",0.110818
81,2014-01-01,"Chicago, IL metro area",0.095487


In [56]:
duckdb.query(
    """

    select
        time,
        region,
        price,
        mean(price) over (partition by region order by time asc rows between 11 preceding and current row) AS avg_sale_price
    from
        us_house_price
        where region = 'Chicago, IL metro area'

"""
).to_df().head(20)

,time,region,price,avg_sale_price
0,2012-01-01,"Chicago, IL metro area",140,140.000000
1,2012-02-01,"Chicago, IL metro area",135,137.500000
2,2012-03-01,"Chicago, IL metro area",150,141.666667
3,2012-04-01,"Chicago, IL metro area",157,145.500000
4,2012-05-01,"Chicago, IL metro area",170,150.400000
5,2012-06-01,"Chicago, IL metro area",181,155.500000
6,2012-07-01,"Chicago, IL metro area",172,157.857143
7,2012-08-01,"Chicago, IL metro area",170,159.375000
8,2012-09-01,"Chicago, IL metro area",160,159.444444
9,2012-10-01,"Chicago, IL metro area",154,158.900000


In [52]:
year_growth = (
    us_house_price.groupby(
        ["Region", us_house_price["Month of Period End"].dt.year]
    )["Median Sale Price"]
    .sum()
    .reset_index()
    .pivot_table(
        index="Region",
        columns="Month of Period End",
        values="Median Sale Price",
        aggfunc=lambda x: x,
    )
)

Month of Period End,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Region,,,,,,,,,,,,,
"Boston, MA metro area",3992000,4306000,4554000,4772000,5016000,5375000,5740000,5949000,6465000,7250000,7787000,8122000,2764000
"Chicago, IL metro area",1895000,2105000,2306000,2476000,2649000,2805000,2909000,2963000,3208000,3579000,3709000,3811000,1318000
"Los Angeles, CA metro area",3932000,4945000,5519000,5923000,6366000,6873000,7346000,7556000,8193000,9598000,10200000,10135000,3492000
National,2186000,2458000,2617000,2797000,2985000,3190000,3367000,3499000,3794000,4454000,4882000,4900000,1668000
"Philadelphia, PA metro area",1839000,1886000,1876000,1935000,2072000,2253000,2382000,2506000,2844000,3127000,3212000,3135000,1063000
"Seattle, WA metro area",3560000,4067000,4368000,4751000,5278000,6052000,6658000,6710000,7298000,8560000,9384000,9178000,3222000
"Washington, DC metro area",3840000,4181000,4311000,4350000,4413000,4580000,4723000,4925000,5330000,5864000,6208000,6359000,2177000


In [17]:
alt.Chart(
    us_house_price.groupby(
        ["Region", us_house_price["Month of Period End"].dt.year]
    )["Median Sale Price"]
    .sum()
    .reset_index(),
    title="",
).mark_bar().encode(
    x=alt.X("Month of Period End:T"), y="Median Sale Price:Q", facet="Region:N"
)

alt.Chart(...)